In [1]:
from random import seed
import numpy as np
import pandas as pd
import json
import os
import sys
import gc
import shutil
from pprint import pprint
from pathlib import Path
from typing import *

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [4]:
import sys

print(str(Path().resolve().parent))
sys.path.append(str(Path().resolve().parent))

from src.utils import (
    seed_every_thing,
    fetch_data,
    Config,
    plot_metric,
    reduce_tf_gpu_memory,
    reduce_mem_usage,
    fetch_custom_data,
    CustomL1Loss
)

/home/yamaguchi-milkcocholate/GoogleBrain-VentilatorPressurePrediction


In [2]:
basedir = Path().resolve().parent
datadir = basedir / "data"
logdir = basedir / "logs"
cachedir = basedir / "cache"

In [6]:
basemodeldir = logdir / "lstm-soft"

In [7]:
cfg_file_path = basemodeldir / "config.json"
with open(cfg_file_path, "rb") as f:
    config = json.load(f)

config = Config().update(config)

In [9]:
train = pd.read_csv(cachedir / "train-10fold-cut-soft-rc-2-debugTrue.csv")
train.shape

(35000, 110)

In [17]:
config.update({"embed_dim": 16})

Config()

In [25]:
def build_model(config: Config, n_features, basemodeldir) -> keras.models.Sequential:
    model = keras.models.Sequential([keras.layers.Input(shape=(config.cut, n_features))])
    for n_unit in config.n_units:
        model.add(
            keras.layers.Bidirectional(
                keras.layers.LSTM(
                    n_unit,
                    return_sequences=True,
                )
            )
        )
    for n_unit in config.n_dense_units:
        model.add(keras.layers.Dense(n_unit, activation="selu"))
    model.add(keras.layers.Dense(1))
    model.load_weights(basemodeldir / "weights_custom_best.h5")
    model.summary()
    
    embed = model.layers[-2].output
    embed = keras.layers.Dense(config.embed_dim, activation="selu")(embed)
    concat = keras.layers.Flatten()(embed)
    outputs = keras.layers.Dense(config.cut)(concat)
    
    fc_model = keras.models.Model(inputs=model.inputs, outputs=outputs)
    fc_model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=config.lr),
        loss='mae')
    return fc_model

In [26]:
fc_model = build_model(config, 110 - 2, basemodeldir / "fold0")

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_36 (Bidirectio (None, 35, 1512)          5231520   
_________________________________________________________________
bidirectional_37 (Bidirectio (None, 35, 1024)          8294400   
_________________________________________________________________
bidirectional_38 (Bidirectio (None, 35, 512)           2623488   
_________________________________________________________________
bidirectional_39 (Bidirectio (None, 35, 256)           656384    
_________________________________________________________________
dense_21 (Dense)             (None, 35, 50)            12850     
_________________________________________________________________
dense_22 (Dense)             (None, 35, 1)             51        
Total params: 16,818,693
Trainable params: 16,818,693
Non-trainable params: 0
__________________________________________

In [28]:
fc_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 35, 108)]         0         
_________________________________________________________________
bidirectional_36 (Bidirectio (None, 35, 1512)          5231520   
_________________________________________________________________
bidirectional_37 (Bidirectio (None, 35, 1024)          8294400   
_________________________________________________________________
bidirectional_38 (Bidirectio (None, 35, 512)           2623488   
_________________________________________________________________
bidirectional_39 (Bidirectio (None, 35, 256)           656384    
_________________________________________________________________
dense_21 (Dense)             (None, 35, 50)            12850     
_________________________________________________________________
dense_23 (Dense)             (None, 35, 16)            816   